# API Data Wrangling Mini Project
## Setup

The API key used the access this project's data is loaded in from an external .env file.

In [2]:
import requests
import os
from dotenv import load_dotenv

#Load the environment.
project_folder = os.path.expanduser('~\Desktop\Springboard')
load_dotenv(os.path.join(project_folder, '.env'))

#Define constants.
API_KEY = os.environ['API_KEY']
URL = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key='

## 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

This section simply collects the requested data by adding API queries to the URL defined above. The JSON data is returned as a string.

In [3]:
start_date = '2017-01-01'
end_date = '2017-12-31'

#Request data from the Franfurt Stock Exchange for the whole year of 2017.
r = requests.get(URL + API_KEY + "&start_date=2017-01-01&end_date=2017-12-31")
json_string = r.json()

## 2. Convert the returned JSON object into a Python dictionary.

Technically, only the first two lines appear to be necessary to solve the problem as written. However, in order to more easily use the data, I went a little farther and made a dictionary keyed by the types of data collected on each trading day from the original dataset, which was keyed by trading days. It may be worth mentioning that the original "json_dict" also contained some metadata, which I don't believe this project required me to use, so I did not include it in the final dictionary.

In [4]:
#Unpack the JSON string to find the column names and data for this dataset.
json_dict = dict(json_string)
json_dict = json_dict['dataset']
col_names = json_dict['column_names']
data = json_dict['data']

#Pivot the data, originally organized so that each row corresponds to  
#the data for one trading day, into lists containing data 
#that correspond to the extracted column names.
data = [[d[r] for d in data] for r in range(len(col_names))]

#Use the column names as keys and the pivoted data as values to create a dictionary.
data_dict = dict(zip(col_names, data))

## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

The list of 'Open' values contains values of 'None' that must be dealt with before we can do anything else. The "remove" function doesn't seem to play well with None as an input, so these value sare converted into the average value of the other elements instead.

Since this was only the first subset of data to be extracted, I assumed that dealing with None values would be a recurring problem and created a function to reference in the future. This turned out not to be the case, but I decided to keep the function anyway because I think the principle for choosing to write it was correct.

In [5]:
#Add all numbers together (skipping None values), then divide by the list's length.
def compute_average(input_list):
    total = 0
    for i in input_list:
        if i is None:
            continue
        total += i
    avg = total/len(input_list)
    return round(avg, 2)

#If the element is a list, leave it alone; otherwise, replace it with the computed average.
def remove_nan_values(input_list):
    avg = compute_average(input_list)
    return [i if type(i) == float else avg for i in input_list]

openings = data_dict['Open']
openings = remove_nan_values(openings)

#Identify and print the highest and lowest values in the list.
max_opening = max(openings)
min_opening = min(openings)
print(max_opening, min_opening)

53.11 34.0


The above cell should print 53.11 for the maximum opening and 34.0 for the minimum opening.

## 4. What was the largest change in any one day (based on High and Low price)?

Since we are only using standard types in this project, a custom-built function for subtracting list values from each other will be helpful. In this project, we will always want the smaller number subtracted from the larger number because we are only interested in the magnitude of the differences, not the direction of the change (positive or negative).

In [6]:
#For each same-index pair, subtract the smaller number from the larger number
#and return a list containing all the computed differences.
def subtract_value_pairs(list1, list2):
    return [abs(i - j) for i,j in zip(list1, list2)]

highs = data_dict['High']
lows = data_dict['Low']

#Take the difference of each pair of values across the lists.
differences = subtract_value_pairs(highs, lows)

#Round to the hundredths place to keep the value consistent with the supplied data,
# and print the value.
max_dif = round(max(differences), 2)
print(max_dif)

2.81


The above cell should print 2.81.

## 5. What was the largest change between any two days (based on Closing Price)?

For this section, I use the same "subtract_value_pairs" function defined above.

In [7]:
closings = data_dict['Close']
#Take the difference of each pair of values across the lists.
differences = subtract_value_pairs(openings, closings)

#Round to the hundredths place and print the value.
max_closing_dif = round(max(differences), 2)
print(max_closing_dif)

1.69


The above cell should print 1.69.

## 6. What was the average daily trading volume during this year?

The function for computing the average from a list of values was defined in the solution to problem 3.

In [8]:
volumes = data_dict['Traded Volume']

#Compute and print the average of the values in the list.
avg = compute_average(volumes)
print(avg)

89124.34


The above cell should print 89124.34.

## 7. What was the median trading volume during this year?

A new function is defined for computing the median.

In [9]:
#This computes the median from a given list of numbers.
def compute_median(input_list):
    sorted_list = sorted(input_list)
    list_len = len(input_list)
    #The index must be an int, so we use floored division to find the "middle" index.
    index = (list_len - 1) // 2

    #If the list length is an odd number 
    #(i.e. dividing by two returns remainder of 1), 
    #return the index computed above.
    if (list_len % 2):
        return sorted_list[index]
    #Otherwise, take the average of the computed index and the next index.
    else:
        return (sorted_list[index] + sorted_list[index + 1])/2.0

#Compute and print the median.
med = compute_median(volumes)
print(med)

76286.0


The above cell should print 76286.0.